# Spark SQL - Transformacje i Analiza Danych

**Cel szkoleniowy:** Opanowanie Spark SQL jako alternatywy dla PySpark DataFrame API

**Zakres tematyczny:**
- Podstawy Spark SQL i rejestracja widokow
- Porownanie skladni SQL vs DataFrame API
- Window Functions w SQL
- CTE (Common Table Expressions) i subqueries
- DDL operacje (CREATE TABLE AS SELECT)

## Kontekst i wymagania

- **Dzien szkolenia**: Dzien 3 - Transformation, Governance, Integrations
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 14.3 LTS+
  - Unity Catalog wlaczony
  - Uprawnienia: CREATE TABLE, CREATE VIEW, SELECT

## Wstep teoretyczny

**Spark SQL vs DataFrame API**

Spark oferuje dwa rownowazne podejscia do przetwarzania danych:

| Aspekt | DataFrame API | Spark SQL |
|--------|---------------|------------|
| Skladnia | Python/Scala | Standard SQL |
| Optymalizacja | Catalyst Optimizer | Catalyst Optimizer |
| Wydajnosc | Identyczna | Identyczna |
| Type Safety | Compile-time | Runtime |
| Integracja | Programistyczna | BI Tools, Analitycy |

**Kiedy uzywac Spark SQL:**
- Analitycy znajacy SQL
- Integracja z narzedzami BI
- Szybkie eksploracje ad-hoc
- Zlozone zapytania z CTE

**Kiedy uzywac DataFrame API:**
- Zlozona logika programistyczna
- Dynamiczne generowanie zapytan
- Reuzywalne komponenty
- Unit testing

## Izolacja per uzytkownik

Uruchom skrypt inicjalizacyjny:

In [ ]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie kontekstu:

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

---

## Czesc 1: Podstawy Spark SQL

### spark.sql() - wykonywanie zapytan SQL

Funkcja `spark.sql()` wykonuje zapytanie SQL i zwraca DataFrame.

**Kluczowe cechy:**
- Zwraca DataFrame (mozna laczyc z DataFrame API)
- Obsluguje wszystkie standardowe operacje SQL
- Wykorzystuje Catalyst Optimizer

In [ ]:
# Przyklad: Proste zapytanie SQL
result = spark.sql("""
    SELECT 
        'Hello Spark SQL' as message,
        current_date() as today,
        current_timestamp() as now
""")

In [ ]:
display(result)

### Tworzenie danych testowych

Przygotujemy dane do demonstracji Spark SQL:

In [ ]:
# Dane zamowien
orders_data = [
    (1, 101, "2024-01-15", 250.00, "completed"),
    (2, 102, "2024-01-16", 150.00, "completed"),
    (3, 101, "2024-01-20", 320.00, "completed"),
    (4, 103, "2024-02-01", 180.00, "pending"),
    (5, 101, "2024-02-10", 420.00, "completed"),
    (6, 102, "2024-02-15", 90.00, "cancelled"),
    (7, 103, "2024-03-01", 550.00, "completed"),
    (8, 104, "2024-03-05", 280.00, "completed"),
    (9, 101, "2024-03-10", 175.00, "completed"),
    (10, 102, "2024-03-15", 340.00, "completed"),
]

orders_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("order_date", StringType(), False),
    StructField("amount", DoubleType(), False),
    StructField("status", StringType(), False)
])

orders_df = spark.createDataFrame(orders_data, orders_schema) \
    .withColumn("order_date", F.to_date("order_date"))

In [ ]:
# Dane klientow
customers_data = [
    (101, "Jan", "Kowalski", "Premium", "Warszawa"),
    (102, "Anna", "Nowak", "Standard", "Krakow"),
    (103, "Piotr", "Wisniewski", "Premium", "Gdansk"),
    (104, "Maria", "Wojcik", "Standard", "Poznan"),
]

customers_schema = StructType([
    StructField("customer_id", IntegerType(), False),
    StructField("first_name", StringType(), False),
    StructField("last_name", StringType(), False),
    StructField("tier", StringType(), False),
    StructField("city", StringType(), False)
])

customers_df = spark.createDataFrame(customers_data, customers_schema)

In [ ]:
display(orders_df.limit(5))

In [ ]:
display(customers_df)

### Rejestracja Temp Views

Aby uzywac DataFrame w zapytaniach SQL, nalezy zarejestrowac je jako widoki tymczasowe.

**Typy widokow:**
- `createOrReplaceTempView()` - widok lokalny dla sesji
- `createOrReplaceGlobalTempView()` - widok globalny (dostepny z `global_temp.nazwa`)

In [ ]:
# Rejestracja widokow tymczasowych
orders_df.createOrReplaceTempView("orders")
customers_df.createOrReplaceTempView("customers")

In [ ]:
# Teraz mozemy uzywac SQL
spark.sql("SELECT * FROM orders LIMIT 5").display()

---

## Czesc 2: Porownanie SQL vs DataFrame API

### Przyklad: Filtrowanie i agregacja

Wykonamy te sama operacje w obu podejsciach.

**Zadanie:** Znajdz laczna wartosc zamowien completed per klient

In [ ]:
# Podejscie DataFrame API
result_df = orders_df \
    .filter(F.col("status") == "completed") \
    .groupBy("customer_id") \
    .agg(
        F.count("*").alias("orders_count"),
        F.sum("amount").alias("total_amount"),
        F.round(F.avg("amount"), 2).alias("avg_amount")
    ) \
    .orderBy(F.col("total_amount").desc())

In [ ]:
display(result_df)

In [ ]:
# Podejscie Spark SQL
result_sql = spark.sql("""
    SELECT 
        customer_id,
        COUNT(*) as orders_count,
        SUM(amount) as total_amount,
        ROUND(AVG(amount), 2) as avg_amount
    FROM orders
    WHERE status = 'completed'
    GROUP BY customer_id
    ORDER BY total_amount DESC
""")

In [ ]:
display(result_sql)

**Porownanie:** Oba podejscia daja identyczny wynik i plan wykonania.

### Przyklad: JOIN z wieloma tabelami

In [ ]:
# DataFrame API - JOIN
joined_df = orders_df \
    .join(customers_df, "customer_id", "inner") \
    .select(
        "order_id",
        F.concat_ws(" ", "first_name", "last_name").alias("customer_name"),
        "tier",
        "order_date",
        "amount",
        "status"
    )

In [ ]:
display(joined_df.limit(5))

In [ ]:
# Spark SQL - JOIN
joined_sql = spark.sql("""
    SELECT 
        o.order_id,
        CONCAT_WS(' ', c.first_name, c.last_name) as customer_name,
        c.tier,
        o.order_date,
        o.amount,
        o.status
    FROM orders o
    INNER JOIN customers c ON o.customer_id = c.customer_id
""")

In [ ]:
display(joined_sql.limit(5))

---

## Czesc 3: Window Functions w SQL

### Skladnia Window Functions

```sql
funkcja() OVER (
    PARTITION BY kolumna
    ORDER BY kolumna
    ROWS BETWEEN ... AND ...
)
```

**Funkcje rankingowe:**
- `ROW_NUMBER()` - unikalny numer wiersza
- `RANK()` - rank z przerwami
- `DENSE_RANK()` - rank bez przerw

**Funkcje analityczne:**
- `LAG()` - wartosc z poprzedniego wiersza
- `LEAD()` - wartosc z nastepnego wiersza
- `FIRST_VALUE()` / `LAST_VALUE()`

In [ ]:
# Ranking zamowien per klient
ranking_sql = spark.sql("""
    SELECT 
        order_id,
        customer_id,
        order_date,
        amount,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date) as order_sequence,
        RANK() OVER (PARTITION BY customer_id ORDER BY amount DESC) as amount_rank,
        DENSE_RANK() OVER (PARTITION BY customer_id ORDER BY amount DESC) as amount_dense_rank
    FROM orders
    WHERE status = 'completed'
    ORDER BY customer_id, order_date
""")

In [ ]:
display(ranking_sql)

### LAG i LEAD - analiza zmian

In [ ]:
# Porownanie z poprzednim zamowieniem
lag_lead_sql = spark.sql("""
    SELECT 
        order_id,
        customer_id,
        order_date,
        amount,
        LAG(amount, 1) OVER (PARTITION BY customer_id ORDER BY order_date) as prev_amount,
        LEAD(amount, 1) OVER (PARTITION BY customer_id ORDER BY order_date) as next_amount,
        amount - LAG(amount, 1) OVER (PARTITION BY customer_id ORDER BY order_date) as amount_change
    FROM orders
    WHERE status = 'completed'
    ORDER BY customer_id, order_date
""")

In [ ]:
display(lag_lead_sql)

### Running Totals i Moving Averages

In [ ]:
# Suma narastajaca i srednia kroczaca
running_sql = spark.sql("""
    SELECT 
        order_id,
        customer_id,
        order_date,
        amount,
        SUM(amount) OVER (
            PARTITION BY customer_id 
            ORDER BY order_date 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) as cumulative_amount,
        ROUND(AVG(amount) OVER (
            PARTITION BY customer_id 
            ORDER BY order_date 
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ), 2) as moving_avg_3
    FROM orders
    WHERE status = 'completed'
    ORDER BY customer_id, order_date
""")

In [ ]:
display(running_sql)

---

## Czesc 4: CTE (Common Table Expressions)

### WITH clause

CTE pozwalaja na tworzenie nazwanych podzapytan, ktore mozna wielokrotnie wykorzystywac.

**Zalety CTE:**
- Czytelnosc kodu
- Reuzywanie logiki
- Latwiejsze debugowanie
- Rekurencyjne zapytania

In [ ]:
# CTE - analiza klientow
cte_analysis = spark.sql("""
    WITH customer_orders AS (
        SELECT 
            customer_id,
            COUNT(*) as orders_count,
            SUM(amount) as total_spent,
            AVG(amount) as avg_order_value
        FROM orders
        WHERE status = 'completed'
        GROUP BY customer_id
    ),
    customer_ranking AS (
        SELECT 
            *,
            RANK() OVER (ORDER BY total_spent DESC) as spending_rank,
            CASE 
                WHEN total_spent >= 500 THEN 'High Value'
                WHEN total_spent >= 300 THEN 'Medium Value'
                ELSE 'Low Value'
            END as value_segment
        FROM customer_orders
    )
    SELECT 
        cr.*,
        c.first_name,
        c.last_name,
        c.tier,
        c.city
    FROM customer_ranking cr
    JOIN customers c ON cr.customer_id = c.customer_id
    ORDER BY spending_rank
""")

In [ ]:
display(cte_analysis)

### Wielokrotne uzycie CTE

In [ ]:
# CTE uzyte wielokrotnie
multi_cte = spark.sql("""
    WITH monthly_stats AS (
        SELECT 
            DATE_TRUNC('month', order_date) as month,
            customer_id,
            SUM(amount) as monthly_spent
        FROM orders
        WHERE status = 'completed'
        GROUP BY DATE_TRUNC('month', order_date), customer_id
    )
    SELECT 
        month,
        COUNT(DISTINCT customer_id) as active_customers,
        SUM(monthly_spent) as total_revenue,
        ROUND(AVG(monthly_spent), 2) as avg_customer_spend,
        MAX(monthly_spent) as max_customer_spend
    FROM monthly_stats
    GROUP BY month
    ORDER BY month
""")

In [ ]:
display(multi_cte)

---

## Czesc 5: Subqueries

### Scalar Subqueries

Podzapytania zwracajace pojedyncza wartosc:

In [ ]:
# Zamowienia powyzej sredniej
scalar_subquery = spark.sql("""
    SELECT 
        order_id,
        customer_id,
        amount,
        (SELECT ROUND(AVG(amount), 2) FROM orders WHERE status = 'completed') as avg_amount,
        amount - (SELECT AVG(amount) FROM orders WHERE status = 'completed') as diff_from_avg
    FROM orders
    WHERE status = 'completed'
      AND amount > (SELECT AVG(amount) FROM orders WHERE status = 'completed')
    ORDER BY amount DESC
""")

In [ ]:
display(scalar_subquery)

### Correlated Subqueries

Podzapytania odnosace sie do zewnetrznego zapytania:

In [ ]:
# Klienci z zamowieniami powyzej ich sredniej
correlated_subquery = spark.sql("""
    SELECT 
        o.order_id,
        o.customer_id,
        o.amount,
        (SELECT ROUND(AVG(o2.amount), 2) 
         FROM orders o2 
         WHERE o2.customer_id = o.customer_id 
           AND o2.status = 'completed') as customer_avg
    FROM orders o
    WHERE o.status = 'completed'
      AND o.amount > (
          SELECT AVG(o2.amount) 
          FROM orders o2 
          WHERE o2.customer_id = o.customer_id 
            AND o2.status = 'completed'
      )
    ORDER BY o.customer_id, o.amount DESC
""")

In [ ]:
display(correlated_subquery)

### EXISTS i IN

In [ ]:
# Klienci ktorzy maja zamowienia > 400
exists_query = spark.sql("""
    SELECT 
        c.customer_id,
        c.first_name,
        c.last_name,
        c.tier
    FROM customers c
    WHERE EXISTS (
        SELECT 1 
        FROM orders o 
        WHERE o.customer_id = c.customer_id 
          AND o.amount > 400
          AND o.status = 'completed'
    )
""")

In [ ]:
display(exists_query)

---

## Czesc 6: CASE WHEN i zaawansowane wyrazenia

### Warunkowa logika

In [ ]:
# Segmentacja zamowien
case_when_sql = spark.sql("""
    SELECT 
        order_id,
        customer_id,
        amount,
        CASE 
            WHEN amount >= 500 THEN 'Large'
            WHEN amount >= 200 THEN 'Medium'
            ELSE 'Small'
        END as order_size,
        CASE status
            WHEN 'completed' THEN 1
            WHEN 'pending' THEN 0
            ELSE -1
        END as status_code,
        COALESCE(amount, 0) as amount_safe
    FROM orders
    ORDER BY amount DESC
""")

In [ ]:
display(case_when_sql)

### NULLIF, COALESCE, NVL

In [ ]:
# Obsluga NULL
null_handling = spark.sql("""
    SELECT 
        order_id,
        amount,
        status,
        NULLIF(status, 'cancelled') as status_or_null,
        COALESCE(NULLIF(status, 'cancelled'), 'N/A') as status_clean,
        NVL(amount, 0) as amount_nvl
    FROM orders
""")

In [ ]:
display(null_handling)

---

## Czesc 7: DDL w Spark SQL

### CREATE TABLE AS SELECT (CTAS)

In [ ]:
# Utworzenie tabeli z wynikami agregacji
spark.sql(f"""
    CREATE OR REPLACE TABLE {CATALOG}.{GOLD_SCHEMA}.customer_summary AS
    SELECT 
        c.customer_id,
        c.first_name,
        c.last_name,
        c.tier,
        c.city,
        COUNT(o.order_id) as total_orders,
        COALESCE(SUM(CASE WHEN o.status = 'completed' THEN o.amount END), 0) as total_spent,
        ROUND(COALESCE(AVG(CASE WHEN o.status = 'completed' THEN o.amount END), 0), 2) as avg_order_value,
        MAX(o.order_date) as last_order_date
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name, c.tier, c.city
""")

In [ ]:
# Weryfikacja
spark.sql(f"SELECT * FROM {CATALOG}.{GOLD_SCHEMA}.customer_summary").display()

### CREATE VIEW

In [ ]:
# Utworzenie widoku
spark.sql(f"""
    CREATE OR REPLACE VIEW {CATALOG}.{GOLD_SCHEMA}.v_monthly_revenue AS
    SELECT 
        DATE_TRUNC('month', order_date) as month,
        COUNT(*) as orders_count,
        SUM(amount) as total_revenue,
        ROUND(AVG(amount), 2) as avg_order_value
    FROM orders
    WHERE status = 'completed'
    GROUP BY DATE_TRUNC('month', order_date)
""")

In [ ]:
spark.sql(f"SELECT * FROM {CATALOG}.{GOLD_SCHEMA}.v_monthly_revenue ORDER BY month").display()

---

## Czesc 8: Explain Plans

### Analiza planu wykonania

EXPLAIN pokazuje jak Spark wykona zapytanie.

In [ ]:
# Plan wykonania dla zlozonego zapytania
spark.sql("""
    EXPLAIN EXTENDED
    SELECT 
        c.customer_id,
        c.first_name,
        SUM(o.amount) as total
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    WHERE o.status = 'completed'
    GROUP BY c.customer_id, c.first_name
    ORDER BY total DESC
""").display()

### Porownanie planow

DataFrame API i SQL generuja identyczny plan:

In [ ]:
# Plan z DataFrame API
df_plan = orders_df \
    .join(customers_df, "customer_id") \
    .filter(F.col("status") == "completed") \
    .groupBy("customer_id", "first_name") \
    .agg(F.sum("amount").alias("total")) \
    .orderBy(F.col("total").desc())

df_plan.explain()

---

## Podsumowanie

### Omowione zagadnienia

1. **Podstawy Spark SQL**
   - `spark.sql()` wykonanie zapytan
   - `createOrReplaceTempView()` rejestracja widokow

2. **Porownanie SQL vs DataFrame API**
   - Identyczna wydajnosc (Catalyst Optimizer)
   - Rozne przypadki uzycia

3. **Window Functions w SQL**
   - ROW_NUMBER, RANK, DENSE_RANK
   - LAG, LEAD
   - Running totals, moving averages

4. **CTE i Subqueries**
   - WITH clause dla czytelnosci
   - Scalar i correlated subqueries
   - EXISTS, IN

5. **DDL operacje**
   - CREATE TABLE AS SELECT
   - CREATE VIEW

---

### Quick Reference

| Operacja | Spark SQL | DataFrame API |
|----------|-----------|---------------|
| Filtrowanie | `WHERE col = 'x'` | `.filter(F.col("col") == "x")` |
| Agregacja | `GROUP BY col` | `.groupBy("col").agg(...)` |
| Ranking | `ROW_NUMBER() OVER (...)` | `row_number().over(window)` |
| CTE | `WITH cte AS (...)` | Brak bezposredniego odpowiednika |
| CASE WHEN | `CASE WHEN ... END` | `F.when(...).otherwise(...)` |

### Nastepne kroki

- **Kolejny notebook**: 03_lakeflow_jobs_orchestration.ipynb
- **Warsztat**: 01_advanced_transformations_workshop.ipynb

---

## Czyszczenie zasobow

In [ ]:
# Usuniecie temp views
spark.catalog.dropTempView("orders")
spark.catalog.dropTempView("customers")

# Opcjonalnie: usuniecie utworzonych tabel
# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{GOLD_SCHEMA}.customer_summary")
# spark.sql(f"DROP VIEW IF EXISTS {CATALOG}.{GOLD_SCHEMA}.v_monthly_revenue")